In [1]:
import time
import random
import requests

In [9]:
class Job104Spider():
    def search(self, keyword, max_nun=10, filter_params=None, sort_type='符合度', is_sort_asc=False):
        # 產生搜尋結果
        jobs = []
        total_count = 0

        url = 'https://www.104.com.tw/jobs/search/list'
        # 這個url不是從網址來的，要到F12>XHR>Headers中查找，所以一定要最後面的list
        query = f'ro=0&kwop=7&keyword={keyword}&expansionType=area,spec,com,job,wf,wktm&mode=s&jobsource=2018indexpoc'
        if filter_params:
            # 加上篩選參數，要先轉換為 URL 參數字串格式
            query += ''.join([f'&{key}={value}' for key, value, in filter_params.items()])
            
        # 加上排序條件
        sort_dict = {
            '符合度': '1',
            '日期': '2',
            '經歷': '3',
            '學歷': '4',
            '應徵人數': '7',
            '待遇': '13',
        }
        
        # 結果排序
        sort_params = f"&order={sort_dict.get(sort_type, '1')}"
        sort_params += '&asc=1' if is_sort_asc else '&asc=0'
        query += sort_params
            
        # 設定headers，偽裝成真人搜尋，避免被網頁偵測到
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
            'Referer': 'https://www.104.com.tw/jobs/search/',
        }
        
        page = 1
        while len(jobs) < max_nun:
            params = f'{query}&page={page}'
            r = requests.get(url, params=params, headers=headers)
            if r.status_code != requests.codes.ok:
                print('請求失敗', r.status_code)
                data = r.json()
                print(data['status'], data['statusMsg'], data['errorMsg'])
                break

            data = r.json()
            total_count = data['data']['totalCount']
            jobs.extend(data['data']['list'])

            if (page == data['data']['totalPage']) or (data['data']['totalPage'] == 0):
                break
            page += 1
            time.sleep(random.uniform(3, 5))

        return total_count, jobs[:max_nun]
    
    def search_job_transform(self, job_data):
        # 轉換格式
        """將職缺資料轉換格式、補齊資料"""
        appear_date = job_data['appearDate']
        apply_num = int(job_data['applyCnt'])
        company_addr = f"{job_data['jobAddrNoDesc']} {job_data['jobAddress']}"

        job_url = f"https:{job_data['link']['job']}"
        job_company_url = f"https:{job_data['link']['cust']}"
        job_analyze_url = f"https:{job_data['link']['applyAnalyze']}"

        job_id = job_url.split('/job/')[-1]
        if '?' in job_id:
            job_id = job_id.split('?')[0]

        salary_high = int(job_data['salaryLow'])
        salary_low = int(job_data['salaryHigh'])

        job = {
            'job_id': job_id,
            'type': job_data['jobType'],
            'name': job_data['jobName'],  # 職缺名稱
            # 'desc': job_data['descSnippet'],  # 描述
            'appear_date': appear_date,  # 更新日期
            'apply_num': apply_num,
            'apply_text': job_data['applyDesc'],  # 應徵人數描述
            'company_name': job_data['custName'],  # 公司名稱
            'company_addr': company_addr,  # 工作地址
            'job_url': job_url,  # 職缺網頁
            'job_analyze_url': job_analyze_url,  # 應徵分析網頁
            'job_company_url': job_company_url,  # 公司介紹網頁
            'lon': job_data['lon'],  # 經度
            'lat': job_data['lat'],  # 緯度
            'education': job_data['optionEdu'],  # 學歷
            'period': job_data['periodDesc'],  # 經驗年份
            'salary': job_data['salaryDesc'],  # 薪資描述
            'salary_high': salary_high,  # 薪資最高
            'salary_low': salary_low,  # 薪資最低
            'tags': job_data['tags'],  # 標籤
        }
        return job
    
    def get_job(self, job_id):
        """get a job content by job_id"""
        url = f'https://www.104.com.tw/job/ajax/content/{job_id}'

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
            'Referer': f'https://www.104.com.tw/job/{job_id}'
        }

        r = requests.get(url, headers=headers)
        if r.status_code != requests.codes.ok:
            print('請求失敗', r.status_code)
            return

        data = r.json()
        return data['data']

In [10]:
job104_spider = Job104Spider()

In [11]:
filter_params = {
    'area': '6001001000,6001016000',  # (地區) 台北市,高雄市
    # 's9': '1,2,4,8',  # (上班時段) 日班,夜班,大夜班,假日班
    # 's5': '0',  # 0:不需輪班 256:輪班
    # 'wktm': '1',  # (休假制度) 週休二日
    # 'isnew': '0',  # (更新日期) 0:本日最新 3:三日內 7:一週內 14:兩週內 30:一個月內
    # 'jobexp': '1,3,5,10,99',  # (經歷要求) 1年以下,1-3年,3-5年,5-10年,10年以上
    # 'newZone': '1,2,3,4,5',  # (科技園區) 竹科,中科,南科,內湖,南港
    # 'zone': '16',  # (公司類型) 16:上市上櫃 5:外商一般 4:外商資訊
    # 'wf': '1,2,3,4,5,6,7,8,9,10',  # (福利制度) 年終獎金,三節獎金,員工旅遊,分紅配股,設施福利,休假福利,津貼/補助,彈性上下班,健康檢查,團體保險
    # 'edu': '1,2,3,4,5,6',  # (學歷要求) 高中職以下,高中職,專科,大學,碩士,博士
    # 'remoteWork': '1',  # (上班型態) 1:完全遠端 2:部分遠端
    # 'excludeJobKeyword': '科技',  # 排除關鍵字
    # 'kwop': '1',  # 只搜尋職務名稱
    }

In [12]:
total_count, jobs= job104_spider.search('python', max_nun=10, filter_params=filter_params)

In [13]:
print('搜尋結果職缺總數：', total_count)

搜尋結果職缺總數： 3387


In [14]:
jobs = [job104_spider.search_job_transform(job) for job in jobs]
print(jobs[:5])

[{'job_id': '7czt1', 'type': '1', 'name': 'Senior Backend Engineer', 'appear_date': '20220105', 'apply_num': 13, 'apply_text': '11~30人應徵', 'company_name': 'GoFreight_聖學科技股份有限公司', 'company_addr': '台北市中山區 民生東路三段2號7樓', 'job_url': 'https://www.104.com.tw/job/7czt1?jobsource=hotjob_chr', 'job_analyze_url': 'https://www.104.com.tw/jobs/apply/analysis/7czt1?channel=104rpt&jobsource=hotjob_chr', 'job_company_url': 'https://www.104.com.tw/company/1a2x6bj764?jobsource=hotjob_chr', 'lon': '121.5376925', 'lat': '25.0576494', 'education': '大學', 'period': '3年以上', 'salary': '年薪2,500,000~4,000,000元', 'salary_high': 2500000, 'salary_low': 4000000, 'tags': ['員工120人', '遠端工作']}, {'job_id': '76bsv', 'type': '1', 'name': '【PropTech數位人才招募】雲端數據工程師', 'appear_date': '20220105', 'apply_num': 4, 'apply_text': '0~5人應徵', 'company_name': '信義房屋股份有限公司', 'company_addr': '台北市信義區 信義路五段100號2樓(110)', 'job_url': 'https://www.104.com.tw/job/76bsv?jobsource=hotjob_chr', 'job_analyze_url': 'https://www.104.com.tw/jobs/apply/an

In [8]:
job_info = job104_spider.get_job('71gqf')
print(job_info)

{'corpImageRight': {'corpImageRight': {'imageUrl': 'https://static.104.com.tw/b_profile/cust_picture/8000/23060248000/custintroduce/image3.jpg?v=20201102190922', 'link': 'https://kad.events.104.com.tw/201023_familymart_rwd/index.html'}}, 'header': {'corpImageTop': {'imageUrl': 'https://static.104.com.tw/b_profile/cust_picture/8000/23060248000/custintroduce/image2.jpg?v=20201102190922', 'link': ''}, 'jobName': '數據分析專員', 'appearDate': '2022/01/05', 'custName': '全家便利商店股份有限公司', 'custUrl': 'https://www.104.com.tw/company/aldh58g', 'applyDate': '', 'analysisType': 1, 'analysisUrl': '//www.104.com.tw/jobs/apply/analysis/71gqf', 'isSaved': False, 'isApplied': False}, 'contact': {'hrName': '張小姐', 'email': '', 'visit': '', 'phone': '', 'other': '', 'reply': '', 'suggestExam': True}, 'environmentPic': {'environmentPic': [{'thumbnailLink': 'https://static.104.com.tw/b_profile/cust_picture/8000/23060248000/env/s_890328564410838386.jpg?v=20201102190922', 'link': 'https://static.104.com.tw/b_profile/